In [38]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
import unicodedata
from ast import literal_eval

In [55]:
df_1 = pd.read_csv("Dataset_Proyecto1\DATA_EDA.csv",  index_col=0)

In [56]:
df1 = df_1[["title", "genres", "overview","popularity" ]]


Configuración de funciones para datos de entrenamiento de MLConseguir una sopa de letras

In [57]:
df1.title = df1.title.str.lower()# convierte todos los títulos a minúsculas
df1.genres = df1.genres.str.replace(" ","")# elimina los espacios en blanco
def safe_literal_eval(value):#función verifica si el valor es una cadena no nula 
    if isinstance(value, str) and not pd.isna(value):#garantiza que no se produzca un error al evaluar los valores nan 
        return literal_eval(value)
    return []
df1.genres = df1.genres.apply(safe_literal_eval)  # Establecer el tipo de datos correcto en la columna genres
df1["features"] = df1.genres  # crea una nueva columna


C:\Users\ROG BY NG\AppData\Local\Temp\ipykernel_1988\467887005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.title = df1.title.str.lower()# convierte todos los títulos a minúsculas
C:\Users\ROG BY NG\AppData\Local\Temp\ipykernel_1988\467887005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.genres = df1.genres.str.replace(" ","")# elimina los espacios en blanco
C:\Users\ROG BY NG\AppData\Local\Temp\ipykernel_1988\467887005.py:7: SettingWithCopyWarning: 
A value is trying to be set on a cop

Conseguir una sopa de letras

In [59]:
df1['genres'] = df1['genres'].apply(lambda x: ' '.join(x))  # Unir elementos de "genres"
df1['features'] = df1['genres'] + ' ' + df1['overview']  # Unir elementos de "genres" y "overview" en la columna "features"(caracteristicas)

C:\Users\ROG BY NG\AppData\Local\Temp\ipykernel_1988\1336136642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['genres'] = df1['genres'].apply(lambda x: ' '.join(x))  # Unir elementos de "genres"
C:\Users\ROG BY NG\AppData\Local\Temp\ipykernel_1988\1336136642.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['features'] = df1['genres'] + ' ' + df1['overview']  # Unir elementos de "genres" y "overview" en la columna "features"(caracteristicas)


In [60]:
df1 = df1.sort_values("popularity", ascending=False)  # Ordenar por popularidad en orden descendente
df1.drop(["genres", "overview", "popularity"], inplace=True, axis=1)  # Eliminar columnas que no se utilizarán en el modelo de ML
df1["features"] = df1["features"].str.lower()  # Normalizar los valores en la columna "features" a minúsculas
df1.reset_index(drop=True, inplace=True)  # Restablecer el índice del DataFrame

In [61]:
df1.head(10)

,title,features
0,minions,f a m i l y a n i m a t i o n a d v e n t ...
1,wonder woman,a c t i o n a d v e n t u r e f a n t a s ...
2,beauty and the beast,f a m i l y f a n t a s y r o m a n c e a ...
3,baby driver,a c t i o n c r i m e after being coerced in...
4,big hero 6,a d v e n t u r e f a m i l y a n i m a t ...
5,deadpool,a c t i o n a d v e n t u r e c o m e d y ...
6,guardians of the galaxy vol. 2,a c t i o n a d v e n t u r e c o m e d y ...
7,avatar,a c t i o n a d v e n t u r e f a n t a s ...
8,john wick,a c t i o n t h r i l l e r ex-lunatic john ...
9,gone girl,m y s t e r y t h r i l l e r d r a m a wi...


Exportar conjunto de datos para datos de entrenamiento de ML

In [62]:
df1.to_csv("API1_fucnion7.csv")#exportar dataset

In [65]:
df = pd.read_csv("API1_fucnion7.csv", index_col=0)

In [66]:
df

,title,features
0,minions,f a m i l y a n i m a t i o n a d v e n t ...
1,wonder woman,a c t i o n a d v e n t u r e f a n t a s ...
2,beauty and the beast,f a m i l y f a n t a s y r o m a n c e a ...
3,baby driver,a c t i o n c r i m e after being coerced in...
4,big hero 6,a d v e n t u r e f a m i l y a n i m a t ...
...,...,...
45340,hellyys,c o m e d y finnish soft erotic movie
45341,altar of fire,d o c u m e n t a r y this film records a 12 d...
45342,vehshi jatt,"classic indian film starring aasia, afzaal ah..."
45343,raakh,after witnessing the public humiliation &amp;...


In [67]:

ml = df.head(10000)  # Utilizar una muestra debido al costo computacional excesivo si se utiliza todo el conjunto de datos
ml.reset_index(drop=True, inplace=True)  # Restablecer el índice del DataFrame 'ml'
ml.reset_index(inplace=True)  # Restablecer el índice nuevamente


In [68]:
# Eliminar filas con valores NaN en la columna "features"
ml = ml.dropna(subset=["features"])

In [69]:
indices = ml[["title", "index"]]  # Obtener un dataset para encontrar el nuevo índice
tfidf = TfidfVectorizer(stop_words="english", max_features=10000)  # Configuración del vector tf-idf, elimina las palabras comunes en inglés y  limita el número de filas a tomar 
tfidf_matrix = tfidf.fit_transform(ml["features"])  # Configuración del vectorizador tf-idf con datos
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)  #Modelo de entrenamiento con los datos proporcionados


In [70]:
def recomendacion(titulo):  # Función de recomendación de la API de ML
    titulo = titulo.lower().strip()  # Convertir el título a minúsculas y eliminar espacios al inicio y al final
    titulo = unicodedata.normalize('NFKD', titulo).encode('ascii', 'ignore').decode('utf-8', 'ignore')  # Filtrar los acentos del título

    idx = indices[indices["title"] == titulo]  # Filtrar el dataset con el título proporcionado

    if idx.empty:  # Verificar si el dataset está vacío
        recommendations = ["Datos no disponible"]
    else:
        idy = idx["index"].iloc[0]  # Obtener el índice
        sim_score = list(enumerate(cosine_sim[idy]))  # Configurar la puntuación de similitud
        sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)  # Ordenar los resultados por puntuación
        sim_score = sim_score[1:6]  # Obtener las 5 películas con mayor puntuación
        movies_index = [i[0] for i in sim_score]  # Obtener los índices de las películas
        recommendations = list(ml['title'].iloc[movies_index].str.title())  # Crear la lista de recomendaciones

    return {'titulo': titulo, 'lista recomendada': recommendations}


In [76]:
titulo="batman"
recomendacion(titulo)

{'titulo': 'batman',
 'lista recomendada': ['Batman Beyond: Return Of The Joker',
  'The Dark Knight Rises',
  'Batman Vs Dracula',
  'Batman: Under The Red Hood',
  'Batman: Mystery Of The Batwoman']}